In [9]:
import argparse
import logging
import os
import jieba
import math
import requests
import wiki as w
from gensim.models.fasttext import FastText
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
from gensim.models import KeyedVectors
fasttext_300d_model = 'pre-trained/cc.zh.300/cc.zh.300.vec'
wordvectors_index = KeyedVectors.load_word2vec_format(fasttext_300d_model)

D:\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
wordvectors_index.most_similar('我')

[('你', 0.7679306864738464),
 ('他', 0.7081897258758545),
 ('想', 0.7019494771957397),
 ('她', 0.6555640697479248),
 ('要', 0.6327636241912842),
 ('真的', 0.6318696737289429),
 ('就', 0.6110949516296387),
 ('知道', 0.6072597503662109),
 ('大家', 0.5957611799240112),
 ('叫我', 0.5952475070953369)]

In [12]:
vec = wordvectors_index['我']
print(vec)
print('The Length of Vector:',len(vec))

[ 3.080e-02  1.008e-01  4.194e-01  2.240e-02  4.070e-02  9.610e-02
  5.750e-02 -5.350e-02  1.923e-01  6.780e-02  1.140e-02  4.450e-02
  4.970e-02  9.500e-03 -1.570e-02 -3.240e-02 -3.530e-02 -6.590e-02
  3.930e-02 -1.124e-01  6.080e-02  6.700e-02  1.300e-02  1.434e-01
 -4.120e-02 -1.507e-01  8.360e-02  1.399e-01 -4.800e-03 -4.170e-02
 -1.082e-01 -5.000e-03  4.000e-04 -9.240e-02 -7.460e-02 -4.210e-02
  7.930e-02 -1.773e-01  1.061e-01 -8.950e-02 -1.600e-02  3.010e-02
 -1.342e-01  2.680e-02 -1.501e-01  1.720e-02  2.620e-02 -1.119e-01
  1.430e-02 -1.890e-02 -1.610e-02 -9.070e-02  2.000e-04 -3.308e-01
  4.670e-02  8.580e-02  1.120e-01  1.470e-02  7.100e-03 -1.120e-01
 -4.690e-02  4.410e-02  1.374e-01  5.790e-02  1.272e-01  1.900e-02
  1.115e-01 -4.380e-02  8.600e-03 -3.690e-02 -3.480e-02  1.228e-01
  1.227e-01  1.112e-01  6.450e-02  3.270e-02  2.500e-02 -4.930e-02
  8.550e-02 -6.020e-02 -9.100e-02  3.880e-02  1.690e-02  2.780e-02
  6.520e-02  5.500e-02 -1.098e-01 -1.470e-01  1.560e-01 -3.289

In [5]:
import pandas as pd
df = pd.read_csv('wikidata.csv')
df

,id,url,title,text
0,13,https://zh.wikipedia.org/wiki?curid=13,數學,數學是利用符號語言研究數量、結構、變化以及空間等概念的一門學科，從某種角度看屬於形式科學的一...
1,18,https://zh.wikipedia.org/wiki?curid=18,哲學,哲學是研究普遍的、根本的問題的學科，包括存在、知識、價值、理智、心靈、語言等領域。哲學與其他...
2,21,https://zh.wikipedia.org/wiki?curid=21,文學,文學在最廣泛的意義上，是任何單一的書面作品。更嚴格地說，文學寫作被認為是一種藝術形式，或被認...
3,22,https://zh.wikipedia.org/wiki?curid=22,歷史,歷史是指人類社會過去的事件和行動，以及對這些事件行為有系統的記錄、詮釋和研究。歷史可提供今人...
4,25,https://zh.wikipedia.org/wiki?curid=25,電腦科學,電腦科學是系統性研究資訊與計算的理論基礎以及它們在電腦系統中如何與應用的實用技術的學科。 它...


In [6]:
df['text'][0]

'數學是利用符號語言研究數量、結構、變化以及空間等概念的一門學科，從某種角度看屬於形式科學的一種。數學透過抽象化和邏輯推理的使用，由計數、計算、量度和對物體形狀及運動的觀察而產生。數學家們拓展這些概念，為了公式化新的猜想以及從選定的公理及定義中建立起嚴謹推導出的定理。'

In [7]:
import jieba
import numpy as np
jieba.set_dictionary('dict.txt.big')

In [13]:
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  

In [53]:
def seg_sentence(sentence):  
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('stops.txt')  # 加載停用詞的路徑  
    outstr = '' 
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += " "   #再次組合成【帶空格】的串
    return outstr

In [55]:
inputs = open('input.txt', 'r', encoding = 'utf-8')
outputs = open('output.txt', 'w')

for line in inputs:  
    line_seg = seg_sentence(line)  # 這裏的返回值是字符串
    outputs.write(line_seg + '\n')

outputs.close()  
inputs.close()  

In [91]:
f = open(r'output.txt')
text = []
for line in f:
    line = line.strip('\n').strip(' ')
    text.append(line)
print(text)
print(text[0].split(' '))

['來到 北京 清華大學', '來到 網易 杭研 大廈', '小明 碩士 畢業 中國科學院 計算所 後 日本京都大學 深造']
['來到', '北京', '清華大學']


In [96]:
import pandas as pd
final = []
for i in range(len(text)):
    x = text[i].split(' ')
    final.append(x)
    df_new = pd.DataFrame(final)
df_new

,0,1,2,3,4,5,6,7
0,來到,北京,清華大學,None,None,None,None,None
1,來到,網易,杭研,大廈,None,None,None,None
2,小明,碩士,畢業,中國科學院,計算所,後,日本京都大學,深造


In [102]:
df_new.iat[0,3]

In [106]:
# 第一列元素個數
df_new.count(axis=1)[0]

3

In [123]:
df_new.count(axis=1)[2]

8

In [135]:
for a in range(0,3):
    for i in range(df_new.count(axis=1)[a]):
        getword = df_new.iat[a,i]
        print('Word:',getword)
        sim = wordvectors_index.most_similar(getword)
        print(sim,'\n')
        vec = wordvectors_index[getword]
        #print(vec)

Word: 來到
[('們到', 0.680351197719574), ('離開', 0.6726597547531128), ('来到', 0.6725727915763855), ('會到', 0.64908766746521), ('帶到', 0.6413955688476562), ('到達', 0.6270354986190796), ('抵達', 0.6244625449180603), ('拜訪', 0.6237394213676453), ('見到', 0.623458981513977), ('剛到', 0.6193416118621826)] 

Word: 北京
[('北京市', 0.695327639579773), ('上海', 0.6890406012535095), ('天津', 0.6707611680030823), ('海淀', 0.6531842947006226), ('丰台', 0.6383175849914551), ('南京', 0.634671688079834), ('深圳', 0.6265233159065247), ('广州', 0.6191778779029846), ('瑞志', 0.6073142290115356), ('成都', 0.6062750816345215)] 

Word: 清華大學
[('國立清華大學', 0.5424829721450806), ('清華', 0.5316276550292969), ('語言所', 0.5306745767593384), ('大學醫學部', 0.5273380279541016), ('本校與', 0.5210415720939636), ('臺北校區', 0.5204195976257324), ('大學教員', 0.5161640644073486), ('學新竹', 0.5138156414031982), ('學賴', 0.5105436444282532), ('學之', 0.5099607110023499)] 

Word: 來到
[('們到', 0.680351197719574), ('離開', 0.6726597547531128), ('来到', 0.6725727915763855), ('會到', 0.64908766746